# Imports

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
data_DB = pd.read_csv("Food_DB.csv")

In [3]:
# Subset of list
data_dinner = data_DB[pd.isnull(data_DB["Lunch"])]
data_lunch = data_DB[data_DB["Lunch"] == "x"]

In [4]:
data_lunch.head()

,Meal,Amount,Stars (1-5),Effort (1-5),Lunch,Ingredients,Notes
6,Fish & Chips,2,4,2,x,NaN,NaN
10,Hotdog,2,5,1,x,NaN,NaN
17,Nudlar,2,3,3,x,NaN,NaN
18,Nuggies & Fries,2,4,1,x,NaN,NaN
21,Plättar,2,5,3,x,NaN,NaN


# Main functions

In [ ]:
# SETTINGS
days_in_plan = 7
starting_meal = 1
current_pos = [starting_meal, 1]
meals_left = 0
current_meal = [None]
leftover_meal_1 = None
leftover_amount_1 = 0
leftover_meal_2 = None
leftover_amount_2 = 0
chosen_meals = []
single_lunch_days = [5]
ready_lunch_days = [3,4]

In [5]:
def pick_random_meal(meal_db, min_amount=0, max_amount=None, limit_selection=True):
    global chosen_meals
    
    if max_amount is not None:
        eligible_meals = meal_db[
            (meal_db['Amount'] >= min_amount) & 
            (meal_db['Amount'] <= max_amount) &
            (~meal_db['Meal'].isin(chosen_meals))
        ]
    else:
        eligible_meals = meal_db[
            (meal_db['Amount'] >= min_amount) &
            (~meal_db['Meal'].isin(chosen_meals))
        ]

    if not eligible_meals.empty:
        random_row = eligible_meals.sample()
        random_meal = random_row.iloc[0]

        current_meal = random_meal
        
        if limit_selection == True:
            chosen_meals.append(current_meal[0])
        
        return current_meal
    else:
        current_meal = "No meal options available."
        return current_meal

In [ ]:
current_meal = pick_random_meal(data_dinner, min_amount=4)
print(current_meal)

In [6]:
def generate_meal_plan(data_lunch, data_dinner, num_days, single_lunch_days, ready_lunch_days):
    meal_plan = pd.DataFrame(index=['lunch', 'dinner'])
    
    global meals_left
    global leftover_meal_1
    global leftover_amount_1
    global leftover_meal_2
    global leftover_amount_2
    
    meal_placeholder = None
    
    # Saves requirements for next lunch
    next_lunch_ready = False
    next_lunch_single = False

    for day in range(1, num_days + 1):
        
        print(f"start of day:{day}")
        print(leftover_meal_1)
        print(leftover_amount_1)
        print(leftover_meal_2)
        print(leftover_amount_2)
        
        
        
        lunch_choice, dinner_choice = None, None
        

        
        # Check next lunch for single or ready made requirement
        if day + 1 in ready_lunch_days:
            next_lunch_ready = True
        else:
            next_lunch_ready = False
        
        if day in single_lunch_days:
            next_lunch_single = True
        else:
            next_lunch_single = False
            
            
            
        # CHECK LEFTOVER STATE (move forward form 2 if 1 empty)
        if leftover_amount_1 <= 0 and leftover_amount_2 != 0:
            leftover_meal_1 = leftover_meal_2
            leftover_amount_1 = leftover_amount_2
            
            leftover_meal_2 = None
            leftover_amount_2 = 0
            
        # LUNCH SELECTION
        
        # Check if there are leftovers, otherwise choose a new lunch from lunch db
        if next_lunch_single == True:
            if leftover_amount_1 == 1:
                lunch_choice = leftover_meal_1
                leftover_amount_1 -= 1

                # At this point leftovers 1 is empty, moving 2 into 1
                leftover_meal_1 = leftover_meal_2
                leftover_amount_1 = leftover_amount_2

                leftover_meal_2 = None
                leftover_amount_2 = 0

            elif leftover_amount_1 > 1:
                lunch_choice = leftover_meal_1
                leftover_amount_1 -= 1
            else:
                lunch_choice = pick_random_meal(data_lunch, min_amount=0, limit_selection=False)["Meal"]
        else:
            if leftover_amount_1 == 1:
                if leftover_amount_2 >= 1:
                    lunch_choice = f"{leftover_meal_1} + {leftover_meal_2}"
                    leftover_amount_1 -= 1
                    leftover_amount_2 -= 1

                    # At this point leftovers 1 is empty, moving 2 into 1
                    leftover_meal_1 = leftover_meal_2
                    leftover_amount_1 = leftover_amount_2

                    leftover_meal_2 = None
                    leftover_amount_2 = 0
                else:
                    lunch_choice = f"{leftover_meal_1} + something"
                    leftover_amount_1 -= 1

            elif leftover_amount_1 == 2:
                lunch_choice = leftover_meal_1
                leftover_amount_1 -= 2

                # At this point leftovers 1 is empty, moving 2 into 1
                leftover_meal_1 = leftover_meal_2
                leftover_amount_1 = leftover_amount_2

                leftover_meal_2 = None
                leftover_amount_2 = 0

            elif leftover_amount_1 > 2:
                lunch_choice = leftover_meal_1
                leftover_amount_1 -= 2
            else:
                lunch_choice = pick_random_meal(data_lunch, min_amount=0, limit_selection=False)["Meal"]
            
        
        # DINNER SELECTION
        
        # In case not ready and not single lunch, pick dinner
        if next_lunch_ready == False:
            if leftover_amount_1 + leftover_amount_2 >=2:
                meal_placeholder = pick_random_meal(data_dinner, min_amount=0, max_amount=2, limit_selection=True)
                dinner_choice = meal_placeholder["Meal"]
                meals_left = meal_placeholder["Amount"]
            
                meals_left -=2
                
            else:  
                meal_placeholder = pick_random_meal(data_dinner, min_amount=0, limit_selection=True)
                dinner_choice = meal_placeholder["Meal"]
                meals_left = meal_placeholder["Amount"]

                meals_left -=2
        elif next_lunch_ready == True:
            if leftover_amount_1 + leftover_amount_2 < 2:
                meal_placeholder = pick_random_meal(data_dinner, min_amount=4, limit_selection=True)
                dinner_choice = meal_placeholder["Meal"]
                meals_left = meal_placeholder["Amount"]
            
                meals_left -=2
            else:
                meal_placeholder = pick_random_meal(data_dinner, min_amount=0, limit_selection=True)
                dinner_choice = meal_placeholder["Meal"]
                meals_left = meal_placeholder["Amount"]
                
                meals_left -=2

        # Update the meal plan
        meal_plan.at['lunch', day] = lunch_choice
        meal_plan.at['dinner', day] = dinner_choice
        
        # At the end of day, check dinner leftovers and move to leftover variables
        if meals_left > 0:
            leftover_meal_2 = dinner_choice
            leftover_amount_2 = meals_left
            
            meals_left = 0
            
        


    return meal_plan

In [7]:
# Example usage:
num_days = 8
single_lunch_days = [0]
ready_lunch_days = [3,4,5,6,7]
chosen_meals = []
leftover_meal_1 = None
leftover_amount_1 = 0
leftover_meal_2 = None
leftover_amount_2 = 0

meal_plan_result = generate_meal_plan(data_lunch, data_dinner, num_days, single_lunch_days, ready_lunch_days)
print(meal_plan_result)
print(chosen_meals)

start of day:1
None
0
None
0
start of day:2
None
0
Sej med tomat
3
start of day:3
Sej med tomat
1
Salt paj
2
start of day:4
Salt paj
1
Curry höna
2
start of day:5
Curry höna
1
Onsdags maletköttsås
3
start of day:6
Onsdags maletköttsås
2
Zucchini pasta
4
start of day:7
Zucchini pasta
4
Hönspesto pasta
3
start of day:8
Zucchini pasta
2
Hönspesto pasta
3
                      1              2                         3  \
lunch   Nuggies & Fries  Sej med tomat  Sej med tomat + Salt paj   
dinner    Sej med tomat       Salt paj                Curry höna   

                            4                                  5  \
lunch   Salt paj + Curry höna  Curry höna + Onsdags maletköttsås   
dinner   Onsdags maletköttsås                     Zucchini pasta   

                           6               7               8  
lunch   Onsdags maletköttsås  Zucchini pasta  Zucchini pasta  
dinner       Hönspesto pasta      Fiskpinnar      Köttbullar  
['Sej med tomat', 'Salt paj', 'Curry höna', 'On